In [1]:
#>>> pd.DataFrame.from_dict(data, orient='index')
%run WebScrapeForLifeExpectancyData.ipynb

['JAPAN', 'SWITZERLAND', 'SPAIN', 'SINGAPORE', 'FRANCE', 'AUSTRALIA', 'CANADA', 'ITALY', 'SOUTH KOREA', 'NORWAY', 'LUXEMBOURG', 'ICELAND', 'SWEDEN', 'ISRAEL', 'NEW ZEALAND', 'AUSTRIA', 'NETHERLANDS', 'IRELAND', 'PORTUGAL', 'MALTA', 'FINLAND', 'UNITED KINGDOM', 'DENMARK', 'BELGIUM', 'GREECE', 'GERMANY', 'SLOVENIA', 'CYPRUS', 'COSTA RICA', 'CHILE', 'CZECH REPUBLIC', 'BAHRAIN', 'CUBA', 'UNITED STATES', 'MALDIVES', 'CROATIA', 'QATAR', 'PANAMA', 'ESTONIA', 'POLAND', 'SLOVAKIA', 'BOSNIA/HERZEG.', 'ARAB EMIRATES', 'URUGUAY', 'OMAN', 'ARGENTINA', 'MONTENEGRO', 'MEXICO', 'ECUADOR', 'BRUNEI', 'CHINA', 'TURKEY', 'ALBANIA', 'ALGERIA', 'VIET NAM', 'SERBIA', 'LEBANON', 'TUNISIA', 'JAMAICA', 'HUNGARY', 'MOROCCO', 'PERU', 'MACEDONIA', 'IRAN', 'BAHAMAS', 'SAINT LUCIA', 'BARBADOS', 'NICARAGUA', 'THAILAND', 'SRI LANKA', 'MALAYSIA', 'ROMANIA', 'HONDURAS', 'BRAZIL', 'COLOMBIA', 'SAMOA', 'LITHUANIA', 'ANTIGUA/BAR.', 'LATVIA', 'BULGARIA', 'ARMENIA', 'MAURITIUS', 'KUWAIT', 'SAUDI ARABIA', 'JORDAN', 'PARAGUAY'

In [31]:
#only load to memory once...
%store -r  WebScrapingData
#countriesAbbrDetails = WebScrapingData
#WebScrapingData


In [35]:
# back up when screwing up with original data operations...
import pickle 
with open ('CountryAbbrCategoryData', 'rb') as file:
    data = pickle.load(file)
WebScrapingData = data

In [189]:
len(WebScrapingData)

183

In [5]:
# New stuff to learn
import pickle 
with open ('ScrapedWebCauseOfDeathData', 'rb') as file:
    data = pickle.load(file)

In [188]:
len(data)

183

In [6]:
import pandas as pd
allData = pd.read_html(data)
#take a long time to load

In [141]:
#deep copy of each dataframe for ease of debugging

processedData = [each.copy() for each in allData]
processedData[0].head(3)

,"INSTRUCTIONS $("".instructions_button"").click(function(){ var aTag = $(""a[name='instructions']""); $('html,body').animate({scrollTop: aTag.offset().top},'slow'); }); Cause of Death","7,405,122,224 WLD","126,451,398 JPN","8,236,303 CHE","48,958,159 ESP","5,888,926 SGP","67,106,161 FRA","23,232,413 AUS","35,623,680 CAN",Unnamed: 9_level_0,Unnamed: 10_level_0,Unnamed: 11_level_0,Unnamed: 12_level_0,Unnamed: 13_level_0,Unnamed: 14_level_0,Unnamed: 15_level_0,Unnamed: 16_level_0
,Rnk,Deaths,Rnk,Deaths,Rnk,Deaths,Rnk,Deaths,Rnk,Deaths,Rnk,Deaths,Rnk,Deaths,Rnk,Deaths,Unnamed: 16_level_1
0,Coronary Heart Disease,1,8727670,1,155766,1,11436,1,53101,2,4901,1,62890,1,23153,1,37876
1,Stroke,2,6221072,3,130159,3,3661,3,28192,4,1806,3,34220,3,10516,5,12693
2,Influenza and Pneumonia,3,3177204,2,145866,8,1865,7,11240,1,5340,5,21096,8,4151,8,6391


In [142]:
worldPopulations = [] 
#x  = processedData
#processedData = processedData[:2]
#clean up the data 
for df, firstTime in zip(processedData, range(1, len(processedData)+1)):
        #print(f"{df.columns.nlevels} column n level")
        #attempts to extract details of countries per data frame
        extractedCountries = [countryName for countryName in df.columns.get_level_values(0)]
        
        #Realign of columns for dropping
        index = [index for index  in df.columns.get_level_values(1)]
        index = [index.pop()] + index
        df.columns = index
        #drop ranking of each disease for each country 
        df.drop(df.columns[1], axis=1, inplace=True)
        #a = df.drop(df.columns[1], axis=1)
        
        #country and population per frame (7?)
        start = (1%firstTime)
        #print(f"{start} > {len(df.columns)}" )
        countriesPopulationsPerFrame =[each.split(" ") for each in extractedCountries[1:len(df.columns)]]
        worldPopulations = worldPopulations + countriesPopulationsPerFrame[start:]
        #print(worldPopulations)
        #becareful of corrupting the dataFrame
        countriesNames = [name[1] for name in countriesPopulationsPerFrame]
        #print(countriesNames)
        df.columns = ['Cause of Death'] + countriesNames





In [143]:
df = pd.concat(processedData, axis=1)
#empty.drop(columns=['WLD'], axis=1)
df = df.loc[:, ~df.columns.duplicated()]
len(df.columns)

183

In [198]:
#WebScrapingData
#worldPopulations = list(set(worldPopulations))
import numpy as np 
worldPopulations 
x = pd.DataFrame(list(zip(*worldPopulations)))
x = x.transpose()
x.columns = ['Population', 'Abbr']
x = x.reindex(columns =['Abbr', 'Population'])

x['Population'] = x['Population'].str.replace(',', '').astype(np.int64)
#x.reset_index(inplace=True)
x.drop_duplicates(inplace=True)
#x.loc[:, :].sort_values(by ="Abbr", ascending=True)
population = x
population.head(3)

,Abbr,Population
0,WLD,7405122224
1,JPN,126451398
2,CHE,8236303


In [108]:
countriesAbbrDetails = WebScrapingData.copy()

In [277]:
countriesAbbrDetails
worldDetails = pd.DataFrame.from_dict(countriesAbbrDetails, orient='index')
#worldDetails['Country'].replace('\t', '')
worldDetails.reset_index(inplace=True)
worldDetails.columns = ['Abbr','Country', 'Category', 'LifeExpectancy']
worldDetails

overAll = pd.merge(worldDetails, population, on=['Abbr'], how='outer')

overAll.loc[overAll['Abbr']=='WLD', 'Country'] = 'WORLD'
overAll.tail(3)
#verAll['Population'] = overAll['Population'].astype(np.int64)
overAll.to_csv('CountriesAbbr_Cat_LifeExp_Pol.csv')
#partial match
worldDetails[worldDetails['Category'].str.contains('developed')]
#worldDetails.to_csv('CountryCat_Popo_Life.')
w = overAll
# = w.astype(str)
#b = w.loc[:, w['Abbr']=='JPN']
#w[w['Abbr']=='JPN']['Country'].item()
w

,Abbr,Country,Category,LifeExpectancy,Population
0,JPN,JAPAN,developed,84.2,1.264514e+08
1,CHE,SWITZERLAND,developed,83.3,8.236303e+06
2,ESP,SPAIN,developed,83.1,4.895816e+07
3,SGP,SINGAPORE,developed,82.9,5.888926e+06
4,FRA,FRANCE,developed,82.9,6.710616e+07
5,AUS,AUSTRALIA,developed,82.9,2.323241e+07
6,CAN,CANADA,developed,82.8,3.562368e+07
7,ITA,ITALY,developed,82.8,6.213780e+07
8,KOR,SOUTH KOREA,developed,82.7,5.118130e+07
9,NOR,NORWAY,developed,82.5,5.320045e+06


In [169]:
df.to_csv("CauseOfDeathPerCountry.csv")
df.head(3)

,Cause of Death,WLD,JPN,CHE,ESP,SGP,FRA,AUS,CAN,ITA,...,CMR,MLI,SWZ,SOM,NGA,CIV,TCD,SLE,CAF,LSO
0,Coronary Heart Disease,8727670,155766,11436,53101,4901,62890,23153,37876,108924,...,12159,8394,732,2841,76410,16622,5921,4464,2685,1339
1,Stroke,6221072,130159,3661,28192,1806,34220,10516,12693,62431,...,14056,8915,662,3923,83379,17644,7044,4757,3899,1258
2,Influenza and Pneumonia,3177204,145866,1865,11240,5340,21096,4151,6391,13823,...,25344,14855,933,19694,305460,32690,28374,10315,6957,1896


In [229]:
x = df.loc[:, ['Cause of Death', 'JPN']].sort_values(by='JPN', ascending=False).head(10)
list(x['Cause of Death'])

['Coronary Heart Disease',
 'Influenza and Pneumonia',
 'Stroke',
 'Lung Cancers',
 'Stomach Cancer',
 'Colon-Rectum Cancers',
 'Lung Disease',
 'Kidney Disease',
 'Pancreas Cancer',
 'Liver Cancer']

In [518]:
# For learning purpose
# multi level indexes
'''
df[0].columns.nlevels
df[0].index.nlevels
'''

r"""
extractData = []
selection = 7 
for eachDataSet in allData:
    df = eachDataSet
    #print(df.columns.get_level_values(0))
    details = [key for key in df.columns.get_level_values(0)]
    [extractData.append(each.split(" ")) for each in details[1:selection+2]]

    #print(extractData)

res1, res2 = map(list, zip(*extractData))

for each in extractData:
    print(each)
"""

'\nextractData = []\nselection = 7 \nfor eachDataSet in allData:\n    df = eachDataSet\n    #print(df.columns.get_level_values(0))\n    details = [key for key in df.columns.get_level_values(0)]\n    [extractData.append(each.split(" ")) for each in details[1:selection+2]]\n\n    #print(extractData)\n\nres1, res2 = map(list, zip(*extractData))\n\nfor each in extractData:\n    print(each)\n'